In [0]:
%run ./0.Config

In [0]:
def Procurement_manager_email(_: str) -> str:
    """
    Sends a professional status update email to the Procurement Manager
    informing them about the completion of the vendor shortlisting process
    and provides a link to the dashboard.
    """
    RECIPIENT = "stalupuri@quantum-i.ai"
    DASHBOARD_LINK = "https://adb-673140172341602.2.azuredatabricks.net/dashboardsv3/01f020cdf48d1c57b8d137cf79234815/published?o=673140172341602"

    llm = ChatDatabricks(endpoint="databricks-llama-4-maverick")

    email_prompt = PromptTemplate(
        input_variables=["recipient_name", "dashboard_link"],
        template="""
You are a professional project coordinator. Write a formal email to the Procurement Manager informing them that the vendor shortlisting process for the Eastern Seaboard region is complete.
Exclude subject from body of the email.
Mention that:
- All vendor submissions have been reviewed.
- Top vendors have been shortlisted based on pricing and other criterias.
- A detailed dashboard has been created to visualize all relevant data and can be accessed here: {dashboard_link}

The email should be addressed to {recipient_name}, written in a professional and courteous tone.

Sign off with:
Best regards,  
Coca Cola Pvt Ltd  
procurement@cocacola.com
""")

    prompt = email_prompt.format(
        recipient_name="Mr. Stalupuri",
        dashboard_link=DASHBOARD_LINK)

    response = llm.invoke(prompt)
    email_body = response.content

    # Send the email
    yag = yagmail.SMTP(EMAIL_USER, EMAIL_PASSWORD)
    yag.send(
        to=RECIPIENT,
        subject="Vendor Shortlisting Completed – Eastern Seaboard",
        contents=email_body)

    print("✅ Email sent to Procurement Manager.")
    return "📩 Status update email successfully sent to the Procurement Manager."

# Wrap it as a LangChain Tool
Procurement_manager_email = Tool(
    name="Procurement_manager_email",
    func=Procurement_manager_email,
    description="Sends an update email to the Procurement Manager with a link to the vendor shortlisting dashboard.",
    return_direct=True
)


In [0]:
tools = [Procurement_manager_email]

agent = initialize_agent(
    tools=tools,
    llm=ChatDatabricks(endpoint="databricks-llama-4-maverick"),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Sends an update email to the Procurement Manager with a link to the vendor shortlisting dashboard.")